In [ ]:
# Toy Model

using Random, Distributions, Optim, NLsolve, SpecialFunctions
N = 2 # Number of countries
periods = 6
J = 3 # Number of goods (needs to be big number and an integer type)
θ = fill(4.0, J) # Frechet shape parameter (governs comparative advantage)

v = 1.0 # migration elasticity
β = .99 # discount factor
α = ones(J) * (1/J) # final good expenditure share

Lt = ones(N, J, periods) # Size of labor force in each country at time 

# assume N, J are defined
τ = fill(1.0, N, N, J)    # 1.1 for all n≠i and goods j

for n in 1:N
    τ[n, n, :] .= 1.0     # set τ[n,n,j]=1.0 for every good j
end

Lt[2,1,1] = 1.2


#At0 = ones(N, J)#initial productivities
At0 = [ 1.0  1.0 1.0
        1.0  1.0 1.0 ]

wt = ones(N, J, periods)
wt0 = ones(N, J)

tradesharest0 = ones(N, N, J) * (1 / N)


μtminus1 = zeros(N, N, J, J) #value of μ at time t = -1
πt0 = ones(N, N, J) #initial trade shares
μt = ones(N, N, J, J, periods)*(1/(N*J))



function tradeSharest0(n, i, j, wt0, At0, τ) #trade shares to nj from ij, equation (7)
    (wt0[i, j] * τ[n, i, j])^(-θ[j]) * At0[i, j] ^θ[j] / (sum((wt0[m, j] *τ[n, m, j])^(-θ[j]) * At0[m, j]^θ[j] for m in 1:N)) 
end


function incomet0(n, j, wt0, Lt) # total income of country n, sector j in time t=0 given wage and labor  
    wt0[n, j] * Lt[n, j, 1] 
end



function Xt0(n, j, α, wt0, Lt) # expenditure on sector good j in region n, from equation (8)
    α[j] * sum(wt0[n, k] * Lt[n, k, 1] for k in 1:J) 
end

#=
function g!(G, wt0)
    for n in 1:N
        for j in 1:J
            G[n, j] = incomet0(n, j, wt0, Lt) - sum(Xt0(i, j, α, wt0, Lt) * tradeSharest0(n, i, j, wt0, At0, τ) for i in 1:N) #market clearing for wt0
        end
    end
end
=#

function g!(G, wt0)
    # Unpack the first N*J entries into a matrix:
    wt0 = reshape(wt0[1:N*J], N, J)
    # Fill your first N*J equations (market clearing)
    idx = 1
    for n in 1:N, j in 1:J
        G[idx] = incomet0(n, j, wt0, Lt) - sum(Xt0(i, j, α, wt0, Lt) * tradeSharest0(n, i, j, wt0, At0, τ) for i in 1:N)
        idx += 1
    end
    # The last equation pins d1[1,1] to 1.0
    G[N*J + 1] = wt0[1,1] - 1.0
end

initial = [1.0, 2.11, 1.2, 1.15, 5.22, 1.21, 1.01]    # size: 2×3 if N=2, J=3
results = nlsolve(g!, initial) #solve for wages

# extract the wage solution as an N×J matrix
wt0 = reshape(results.zero[1:N*J], N, J)

wt[:,:, 1] .= wt0[:,:]

for n in 1:N
    for i in 1:N
        for j in 1:J
            tradesharest0[n, i , j] = tradeSharest0(n, i, j, wt0, At0, τ)
        end
    end
end

println(wt0)  #end solving for wt0

display(tradesharest0)

In [ ]:
# Toy Model: Back to Basics

using Random, Distributions, Optim, NLsolve, SpecialFunctions
N = 3 # Number of countries
periods = 1
J = 1 # Number of goods (needs to be big number and an integer type)
θ = fill(4.0, J) # Frechet shape parameter (governs comparative advantage)

v = 1.0 # migration elasticity
β = .99 # discount factor
α = ones(J) * (1/J) # final good expenditure share

Lt = ones(N, J, periods) # Size of labor force in each country at time 

# assume N, J are defined
τ = fill(1.0, N, N, J)    # 1.1 for all n≠i and goods j

for n in 1:N
    τ[n, n, :] .= 1.0     # set τ[n,n,j]=1.0 for every good j
end



#At0 = ones(N, J)#initial productivities
At0 = [ 1.0; 1.1; 1.0]


wt = ones(N, J, periods)
wt0 = ones(N, J)

tradesharest0 = ones(N, N, J) * (1 / N)


μtminus1 = zeros(N, N, J, J) #value of μ at time t = -1
πt0 = ones(N, N, J) #initial trade shares
μt = ones(N, N, J, J, periods)*(1/(N*J))



function tradeSharest0(n, i, j, wt0, At0, τ) #trade shares to nj from ij, equation (7)
    (wt0[i, j] * τ[n, i, j])^(-θ[j]) * At0[i, j] ^θ[j] / (sum((wt0[m, j] *τ[n, m, j])^(-θ[j]) * At0[m, j]^θ[j] for m in 1:N)) 
end


function incomet0(n, j, wt0, Lt) # total income of country n, sector j in time t=0 given wage and labor  
    wt0[n, j] * Lt[n, j, 1] 
end


function Xt0(n, j, α, wt0, Lt) # expenditure on sector good j in region n, from equation (8)
    α[j] * sum(wt0[n, k] * Lt[n, k, 1] for k in 1:J) 
end

#=
function g!(G, wt0)
    for n in 1:N
        for j in 1:J
            G[n, j] = incomet0(n, j, wt0, Lt) - sum(Xt0(i, j, α, wt0, Lt) * tradeSharest0(n, i, j, wt0, At0, τ) for i in 1:N) #market clearing for wt0
        end
    end
end
=#

function g!(G, wt0)
    # Unpack the first N*J entries into a matrix:
    wt0 = reshape(wt0[1:N*J], N, J)
    # Fill your first N*J equations (market clearing)
    idx = 1
    for n in 1:N, j in 1:J
        G[idx] = incomet0(n, j, wt0, Lt) - sum(Xt0(i, j, α, wt0, Lt) * tradeSharest0(n, i, j, wt0, At0, τ) for i in 1:N)
        idx += 1
    end
    # The last equation pins d1[1,1] to 1.0
    G[N*J + 1] = wt0[1,1] - 1.0
end

initial = [1.0, 2.11, 1.2, 1.15, 5.22, 1.21, 1.01]    # size: 2×3 if N=2, J=3
results = nlsolve(g!, initial) #solve for wages

# extract the wage solution as an N×J matrix
wt0 = reshape(results.zero[1:N*J], N, J)

wt[:,:, 1] .= wt0[:,:]

for n in 1:N
    for i in 1:N
        for j in 1:J
            tradesharest0[n, i , j] = tradeSharest0(n, i, j, wt0, At0, τ)
        end
    end
end

println(wt0)  #end solving for wt0

display(tradesharest0)


In [ ]:
using Random, Distributions, Optim, NLsolve, SpecialFunctions

N = 3 # Number of countries
θ = 4 # Frechet shape parameter (governs comparative advantage)
T = ones(N, 1) # Frechet scale parameter (governs absolute advantage)
σ = 2 # Substitution elasticity between goods
τ = ones(N, N) # Iceberg trade costs
L = ones(N, 1) # Size of labor force in each country
T[1] = 5.0

w = ones(Float64, N) #creating an array of N countries' wages initialized with one
w[1] = 1 #assigning country one's wage as 1
#print(w[1])
#print(" ")
#print(w[2])



#the real step 2:
function ϕ(i) #price parameter for country i
    h = 0 #helps in the calculation below
    for z = 1:N
        h = h + T[z] * (w[z] * τ[i, z])^-θ #equation (7) from Eaton-Kortum
    end
    return h
end


#the not real step 2:
function tradeShares(n, i) #computes fraction of goods country n buys from country i
    (T[i] * (w[i] * τ[n, i])^-θ) / ϕ(n) #equation (8) from Eaton-Kortum
end

function printTradeShares(n, i) #prints the trade shares function
    print("fraction of goods country ")
    print(n)
    print(" buys from country ")
    print(i)
    println(":")
    println(tradeShares(n, i))
    return nothing
end


#what we were supposed to do:
function Y(i, wage) #calculates total income in country i
    wage * L[i]
end


#NOT what we were supposed to do:
biTradeBalance = ones(Float64, N, N) #an array storing the bilateral trade balance between countries n and i

function Y(i) #calculates total income in country i
    w[i] * L[i]
end
function exports(n, i) #calculates the exports country n sells to country i
    tradeShares(i, n) * Y(i)
end
function imports(n, i) #calculates the imports country n buys from country i
    tradeShares(n, i) * Y(n)
end

sol = zeros(Float64, N) #the output of the optimization
welfare = zeros(Float64, N) #an array to store equilibrium welfare
perCapitaWelfare = zeros(Float64, N) #an array to store equilibrium welfare
gam = (gamma((θ + 1 - σ) / θ))^(1 / (1 - σ)) #a parameter in the price index function
function priceIndex(i) #the price level in country i
    gam * (ϕ(i))^(-1 / θ)
end
function wageWelfare(i) #calculating equilibrium wage and welfare of country i, storing them in their respective arrays
    w[i] = sol[i] #stores country i's equillibrium wage in the wage array
    welfare[i] = (sol[i] * L[i]) / priceIndex(i) #calculating and storing equilibrium welfare for country i (total income / price level)
    perCapitaWelfare[i] = sol[i] / priceIndex(i) #calculating and storing the equilibrium pre-capita welfare for country i
    print("country ")
    print(i)
    print("'s wage: ")
    println(w[i])
    print("welfare: ")
    println(welfare[i])
    print("welfare per capita: ")
    println(perCapitaWelfare[i])
    return nothing
end
    

#continuum of goods:
function f!(F, w) #the system of equations to optimize
    F[1] = (((T[1] * (1 * τ[1, 1])^-θ) / ϕ(1)) * 1 * L[1]) - (((T[1] * (1 * τ[1, 1])^-θ) / ϕ(1)) * 1 * L[1]) + sum((((T[1] * (1 * τ[i, 1])^-θ) / ϕ(i)) * w[i] * L[i]) - (((T[i] * (w[i] * τ[1, i])^-θ) / ϕ(1)) * 1 * L[1]) for i in 2:N)
    for n = 2:N
        F[n] = (((T[n] * (w[n] * τ[1, n])^-θ) / ϕ(1)) * 1 * L[1]) - (((T[1] * (1 * τ[n, 1])^-θ) / ϕ(n)) * w[n] * L[n]) + sum((((T[n] * (w[n] * τ[i, n])^-θ) / ϕ(i)) * w[i] * L[i]) - (((T[i] * (w[i] * τ[n, i])^-θ) / ϕ(n)) * w[n] * L[n]) for i in 2:N)
    end
end
results = nlsolve(f!, [1.0; 1.1; 1.2]) #solving for wages with country 1 set at 1.0
for i = 1:N
    sol[i] = results.zero[i] #saving results in sol[i]
    #println(sol[i])
end
for j = 1:N
    wageWelfare(j)
end

for n = 1:N
    for i = 1:N
        printTradeShares(n, i)
    end
end

display(w)